In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import gradio as gr
from diffusers import QwenImagePipeline
from packaging import version


# =====================================================
# Device & dtype
# =====================================================
if torch.cuda.is_available():
    device = "cuda"
    torch_dtype = torch.bfloat16
else:
    device = "cpu"
    torch_dtype = torch.float32

print(f"[INFO] Using device: {device}")
print(f"[INFO] Gradio version: {gr.__version__}")


# =====================================================
# Load Qwen Image pipeline (ONCE)
# IMPORTANT: with device_map, DO NOT call pipe.to(...)
# =====================================================
pipe = QwenImagePipeline.from_pretrained(
    ".././qwen_image_get_model/Qwen-Image-2512",
    torch_dtype=torch_dtype,
    device_map="balanced",
    max_memory={
        0: "80GiB",
        "cpu": "120GiB"
    },
    low_cpu_mem_usage=True,
    local_files_only=True
)

# Optional (runtime memory helpers)
# pipe.enable_attention_slicing()
# pipe.enable_xformers_memory_efficient_attention()


# =====================================================
# Aspect ratios
# =====================================================
ASPECT_RATIOS = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1104),
    "3:4": (1104, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}


# =====================================================
# Generation function
# =====================================================
def generate_image(prompt, negative_prompt, aspect_ratio, steps, cfg_scale, seed):
    width, height = ASPECT_RATIOS[aspect_ratio]

    gen_device = "cuda" if torch.cuda.is_available() else "cpu"
    generator = torch.Generator(device=gen_device)

    seed = int(seed)
    if seed >= 0:
        generator.manual_seed(seed)
    else:
        generator = None  # random

    with torch.inference_mode():
        out = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            width=width,
            height=height,
            num_inference_steps=int(steps),
            true_cfg_scale=float(cfg_scale),
            generator=generator,
        )

    return out.images[0]


# =====================================================
# Gradio UI
# =====================================================
with gr.Blocks(title="Qwen Image Generator") as demo:
    gr.Markdown("## Qwen Image Generator (Diffusers)")
    gr.Markdown(f"**Device:** `{device}`")

    with gr.Row():
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                lines=8,
                label="Prompt",
                value=(
                    "A 20-year-old East Asian girl with delicate, charming features and "
                    "large, bright brown eyes—expressive and lively, with a cheerful or subtly smiling expression. "
                    "Her naturally wavy long hair is either loose or tied in twin ponytails. "
                    "She has fair skin and light makeup accentuating her youthful freshness. "
                    "She wears a modern, cute dress or relaxed outfit in bright, soft colors. "
                    "She stands indoors at an anime convention, casual iPhone snapshot."
                ),
            )
            negative_prompt = gr.Textbox(
                lines=4,
                label="Negative Prompt",
                value="低分辨率，低画质，肢体畸形，手指畸形，画面过饱和，蜡像感，人脸无细节，过度光滑，画面具有AI感。构图混乱。文字模糊，扭曲。",
            )

        with gr.Column(scale=1):
            aspect_ratio = gr.Dropdown(list(ASPECT_RATIOS.keys()), value="16:9", label="Aspect Ratio")
            steps = gr.Slider(5, 50, value=10, step=1, label="Inference Steps")
            cfg = gr.Slider(1.0, 8.0, value=4.0, step=0.1, label="CFG Scale")
            seed = gr.Number(value=42, precision=0, label="Seed (-1 = random)")
            generate = gr.Button("Generate")

    output = gr.Image(label="Generated Image", type="pil")

    generate.click(
        fn=generate_image,
        inputs=[prompt, negative_prompt, aspect_ratio, steps, cfg, seed],
        outputs=output,
    )

# =====================================================
# Queue (version-safe)
# =====================================================
try:
    # Gradio 4.x
    if version.parse(gr.__version__) >= version.parse("4.0.0"):
        demo.queue(default_concurrency_limit=1)
    else:
        # Gradio 3.x
        demo.queue(concurrency_count=1)
except TypeError:
    # Fallback for versions that accept no args
    demo.queue()

# Launch
demo.launch(share=True)


[INFO] Using device: cuda
[INFO] Gradio version: 6.2.0


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://279259b4bbfc9de12c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]